In [1]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras 
from tensorflow.keras.models import Model
from ModuleLibrary.metrics import BA, MCC
from ModuleLibrary.plots import Features_exploration, Chromosome_prediction
from ModuleLibrary.utils import encode_aa, load_data_prot, one_hot_prot, traduction, sliding_window_view
import os

2022-04-19 09:57:53.443993: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [ ]:
# KEEP
bug = 0
test = h5py.File('Uniprot/data.hdf5','w')
with open('Uniprot/uniprot_sprot.fasta', 'r') as f:
    res = None
    id = 0
    for line in f:
        if line.startswith('>'):
            if res is not None:
                res = ''.join(res)
                res = np.array(list(res))
                try:
                    data = encode_aa(res)
                except:
                    res = []
                    bug += 1
                    print(bug, id)
                    continue
                test.create_dataset(str(id), data=data)
                id += 1
            res = []
        else:
            res.append(line.strip())
test.close()

In [2]:
#KEEP
win = 30
db = h5py.File('Uniprot/data_sprot.hdf5','r')
data = []
size = len(db.keys())
for i, key in enumerate(db.keys()):
    print(f"In progress : {round(i/size*100,2)}%", end = "\r")
    prot = np.array(db[key])
    prot = prot[:len(prot)-len(prot)%win]
    prot = one_hot_prot(prot)
    prot = prot.reshape((-1,win,21))
    data.append(prot)
    
data = np.concatenate(data)

np.save(f'Uniprot/data_sprot_oh_w{win}.npy', data)

In [ ]:
# KEEP
files = os.listdir(f'Data/DNA/HS37/hdf5')
for i, f in enumerate(files):
    print(f'{i+1}/{len(files)}')
    h5 = h5py.File(f'Data/DNA/HS37/hdf5/{f}','r')
    chr = np.array(h5['data']).astype('int8')import numpy as np

lab = np.zeros(100, dtype='int8')
print(lab)
def fill(x,y):
    for i in range(x,y+1):
        lab[i] = 1

a = np.array([3, 50, 90])
b = np.array([9, 55, 99])

fill_vec = np.frompyfunc(fill, 2,0)
fill_vec(a, b)
print(lab)
    chr = chr.reshape(chr.shape[0])
    prot = traduction(chr)
    prot_oh = one_hot_prot(prot)
    print(prot_oh.dtype)
    np.save(f"Data/DNA/HS37/prot_OH/{f.replace('hdf5','')}npy", prot_oh)
    

In [7]:
win = 30
h5 = h5py.File(f'Data/DNA/HS37/hdf5/chr1.hdf5','r')
dna = np.array(h5['data']).astype('int8')
dna = dna.reshape(dna.shape[0])
dna = dna[:100000]
dna = np.append(np.zeros(((win*3)//2),dtype='int8'), dna, axis=0)
dna = np.append(dna, np.zeros(((win*3)//2),dtype='int8'), axis=0)
data = sliding_window_view(dna, win*3)
print(data.shape)
data = np.apply_along_axis(traduction, axis=1, arr=data)
print(data.shape)
data = np.apply_along_axis(one_hot_prot, axis=1, arr=data)
print(data.shape)
np.save('Uniprot/HS37_chr1_allFrame_OH.npy',data)

(100001, 90)
(100001, 30)
(100001, 30, 21)
